In [0]:
spark.conf.set(
  "fs.azure.account.key.projectnf.dfs.core.windows.net",
  "ntzj8hOFCbU6O7L069dR4Bxqnb/uE8v9STC+eqHFitEANjJpSskZE0oW9GI6o3xr0TP5AOagZb0P+AStupAZ8g=="
)

In [0]:
df = spark.read.csv(
    "abfss://lakehouse@projectnf.dfs.core.windows.net/processed/metadata_csv/",
    header=True,
    inferSchema=True
)

df.write.mode("overwrite").parquet(
    "abfss://lakehouse@projectnf.dfs.core.windows.net/processed/metadata_parquet/"
)

In [0]:
metadata = spark.read.parquet(
    "abfss://lakehouse@projectnf.dfs.core.windows.net/processed/metadata_parquet/"
)

metadata.show(5)

+--------------------+--------------------+----+----+
|            filename|               label|path|size|
+--------------------+--------------------+----+----+
|+ 7W71+ 7� 7...|                NULL|NULL|NULL|
|                NULL|9g96� 99 99r9...|NULL|NULL|
|R \b  �0 ...|                NULL|NULL|NULL|
|                  J�|                NULL|NULL|NULL|
| 9�>J> 9�1\bJ1\...|                NULL|NULL|NULL|
+--------------------+--------------------+----+----+
only showing top 5 rows


In [0]:
from pyspark.sql.functions import col

df = metadata

# 1. Remove rows with missing important fields
df = df.filter(
    col("filename").isNotNull() &
    col("label").isNotNull() &
    col("path").isNotNull()
)

# 2. Remove images with zero or negative size
df = df.filter(
    col("size").cast("int") > 0
)

df.show(5)
df.count()


+--------+-----+--------------------+-----+
|filename|label|                path| size|
+--------+-----+--------------------+-----+
| Y52.jpg|  yes|abfss://lakehouse...|61240|
| Y53.jpg|  yes|abfss://lakehouse...| 5011|
| Y54.jpg|  yes|abfss://lakehouse...|15874|
| Y55.jpg|  yes|abfss://lakehouse...|43087|
| Y56.jpg|  yes|abfss://lakehouse...| 7633|
+--------+-----+--------------------+-----+
only showing top 5 rows


506

In [0]:
from pyspark.sql.functions import when

df = df.withColumn(
    "label_index",
    when(col("label") == "yes", 1).otherwise(0)
)

In [0]:
df.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@projectnf.dfs.core.windows.net/gold/brain_metadata_clean/"
)


In [0]:
df.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@projectnf.dfs.core.windows.net/gold/brain_metadata_clean/"
)
